In [390]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *
from _rulesbuilding import *

In [391]:
rules = pd.read_csv(config.files['service_rules'])

In [392]:
rules = rules[(rules['text_match'].notna()) & (rules['text_exclude'].notna() == False)]

In [393]:
regexer = GroupedDataRegex('|')

matchers = []
rdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'M', '(', ')'))
sdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'S', '^(', ')$'))
adecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'A', '^.*(', ').*$'))

regexes = {}
for matcher in matchers:
    type = rules['matching']
    regex = rules[rules['matching'] == matcher.getType()][['service_id','text_match']]
    regex = matcher.make(regex.groupby('service_id')['text_match']).to_dict()
    inv_map = {v: k for k, v in regex.items()}
    regexes = {**regexes, **inv_map}
    

# output(regexes)

In [394]:
signals = {'^.*(' + ('|'.join(v)) + ').*$' : k for k, v in config.signals.items()}

output(signals)

{'^.*(cancelled|cancel).*$': 'cancellation',
 '^.*(coming|back|signup|signing|joining|welcome).*$': 'signup'}


In [395]:
data = pd.read_csv(config.files['raw'])

In [396]:
data['service_id'] = data[['description']]
data['service_id'] = data[['service_id']].replace({'service_id':regexes}, regex=True)

In [397]:
data['signal_type'] = data[['description']]
data['signal_type'] = data[['signal_type']].replace({'signal_type': signals}, regex = True)

print(data.to_string())

        item_id    buyer_id  order_date  merchant_id                  merchant_name status      last_updated                                                                                         description                                                                                          service_id                                                                                         signal_type
0    1191248928 -1039890773  2016-02-19            6                        Netflix      N  2016-02-20 11:18                                                                  Thanks for coming back to Netflix!                                                                                                   3                                                                                              signup
1    1191248937 -1039890773  2016-02-19            6                        Netflix      N  2016-02-20 11:18         we've cancelled your Netflix Account. This change will be effecti